# Dependencies



In [1]:
%cd drive/MyDrive/LLMs/wikipedia

/content/drive/MyDrive/LLMs/wikipedia


In [2]:
!pip install datasets
!pip install textstat
!pip install wandb
#!pip install -U sentence-transformers
!pip install apache-beam
!pip install transformers[torch] evaluate
!pip install mwparserfromhell
!pip install openai
!pip install wandb
!pip install tiktoken

  Using cached dill-0.3.1.1-py3-none-any.whl
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.15 requires dill>=0.3.7, but you have dill 0.3.1.1 which is incompatible.
  Using cached dill-0.3.7-py3-none-any.whl (115 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.1.1
    Uninstalling dill-0.3.1.1:
      Successfully uninstalled dill-0.3.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.49.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.


In [3]:
from datasets import load_dataset, load_from_disk, DatasetDict
#from sentence_transformers import SentenceTransformer
from textstat import flesch_reading_ease as fre
from textstat import flesch_kincaid_grade as fkg
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import re

import os
import openai
import json
from getpass import getpass

from scipy.stats import kstest
import numpy as np
import wandb
import datetime
from datetime import datetime
from collections import defaultdict
from tqdm import tqdm
from time import time
from time import sleep

from itertools import product
import tiktoken

In [4]:
wandb.login()

wandb: Currently logged in as: dmeltzer (ft-llmmm). Use `wandb login --relogin` to force relogin


True

# Definitions

In [5]:
def flesch_scores(example):
    """
    Calculate Flesch Readability scores for each article
    """

    example['fre'] = fre(example['text'])
    example['fkg'] = fkg(example['text'])
    return example

def remove_text_between_curly_braces(text):

    pattern = r"{{(.|\n)*?}}"
    text = re.sub(pattern, "", text)

    pattern = r"{(.|\n)*?}"
    text = re.sub(pattern, "", text)
    return text


def extract_article_text_V2(example):
    """
    Extracts the text from a Wikipedia article before the "Related pages" and "References" sections.

    Parameters:
        example (dict): Dictionary with key 'text' which contains entire Wikipedia article content as a string.

    Returns:
        str: The article text before the "Related pages" and "References" sections.
    """
    text = example['text']

    # Find the positions of the "Related pages" and "References" sections
    txt_lower = text.lower()

    related_pages_position = txt_lower.find('related pages')
    references_position = txt_lower.find('references')

    # Extract the text before the sections
    article_text = ""
    if related_pages_position != -1:
        # If the "Related pages" section is found, extract the text before it
        article_text = text[:related_pages_position]
    elif references_position != -1:
        # If the "References" section is found (but not the "Related pages" section),
        # extract the text before it
        article_text = text[:references_position]
    else:
        # If both sections are not present, return the entire text
        article_text = text

    article_text = article_text.replace('  ',' ')
    article_text = remove_text_between_curly_braces(article_text)

    example['text'] = article_text

    return example

class trunc_article_wrapper:
    def __init__(self, max_length=300):
        self.max_length = max_length

    def trunc_article_func(self, example):

        text = example['text']
        paragraphs = text.split('\n\n')

        trunc_text = paragraphs[0]
        i=1

        while len(trunc_text)<=300 and i<len(paragraphs):
            if len(paragraphs[i].split())>=5:
                trunc_text += ' ' + paragraphs[i]
            i += 1

        if '.' in trunc_text:
            idx = trunc_text[::-1].index('.')
            trunc_text = trunc_text[:len(trunc_text)-idx]

        example['trunc_text'] = trunc_text
        return example

def trunc_article(dataset,max_length=300):
    f = trunc_article_wrapper(max_length).trunc_article_func

    dataset = dataset.map(lambda article: f(article))
    return dataset

def system_message(message,extra=None):
    if extra is None:
        message = 'You are a helpful assistant that generates instructions from text.'
    else:
        message = f'You are a helpful assistant that generates instructions from text. {extra}'

def create_prompt(prompt_start,text,prompt_end):

    prompt = f"{prompt_start} {text} {prompt_end}"
    return prompt

def GPT_custom_prompt(message,
                      prompt_start,
                      text,
                      prompt_end,
                      model_engine ='gpt-3.5-turbo'):

    prompt = create_prompt(prompt_start,
                           text,
                           prompt_end)

    response=openai.ChatCompletion.create(
        model=model_engine,
        messages=[
            {"role": "system", "content": message},
            {"role": "user", "content": prompt},
        ],
        temperature = 0)

    instruction = response['choices'][0]['message']['content']

    return instruction

def make_instruct_dataset(dataset,
                          message,
                          prompt_template,
                          split='train',
                          text_key='trunc_text',
                          length=10**3,
                          sleep_timer=1,
                          file_name='df_wiki_questions'):

    output_file = f'./data/{file_name}_{split}.csv'

    if os.path.exists(output_file):
        print('reloading file')
        df = pd.read_csv(output_file,
                         index_col='Unnamed: 0')

        prev_length = len(df)

        print(f'length of dataset is {prev_length}')

    else:
        df = pd.DataFrame(columns=['id',
                                   'system_message',
                                   'prompt_template',
                                   'question'])
        prev_length = 0

    ds = dataset[split]

    for i in tqdm(range(length)):
        row = ds[i]

        if int(row['id']) in set(df['id']):
            continue

        temp_dict = {}
        temp_dict['id'] = row['id']
        temp_dict['system_message'] = message
        temp_dict['prompt_template'] = prompt_template
        temp_dict['question'] = GPT_custom_prompt(message,
                                                  prompt_template[0],
                                                  row[text_key],
                                                  prompt_template[1])
        sleep(sleep_timer)

        df.loc[len(df)] = temp_dict
        df.to_csv(output_file)

        print(f'\nlength of dataset is {len(df)}')

# Filtering Dataset

In [ ]:
simple_wiki = load_dataset("wikipedia", "20220301.simple")

simple_wiki_articles = simple_wiki.map(extract_article_text_V2)

simple_wiki_trunc = trunc_article(simple_wiki_articles)
simple_wiki_trunc = simple_wiki_trunc.filter(lambda x:50<=len(x['trunc_text'].split())<=500)

simple_wiki_scored = simple_wiki_trunc.map(flesch_scores)

In [45]:
simple_wiki_filt = simple_wiki_scored.filter(lambda article: article['fre']>=60
                                             and article['fkg']<9)

simple_wiki_filt = simple_wiki_filt.filter(lambda article:
                                           not any(spec_chr in article['trunc_text']
                                                   for spec_chr in {'{{','class=',
                                                                    'infobox'}))

Filter:   0%|          | 0/75310 [00:00<?, ? examples/s]

In [65]:
simple_wiki_split_long = DatasetDict()

split_dataset_tmp = simple_wiki_filt['train'].train_test_split(5000,seed=42)

simple_wiki_split_long=split_dataset_tmp['train'].train_test_split(5000,seed=42)
simple_wiki_split_long['validation'] = split_dataset_tmp['test']

simple_wiki_split_long.save_to_disk('./data/simple_wiki_split_long')

Saving the dataset (0/1 shards):   0%|          | 0/65305 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [66]:
now = datetime.now()
file_name = './data/simple_wiki_split_long'
time_stamp = now.strftime("%m.%d.%y-%H.%M.%S")
with wandb.init(project='ELI5_analysis',
                            entity='ft-llmmm',
                            job_type='log_data',
                            name=f'wiki_long_articles_{time_stamp}') as run:
                # Initialize a WandB run for logging
                data_art = wandb.Artifact('simple_wiki_split_long', 'dataset')
                data_art.add_dir(file_name)
                run.log_artifact(data_art)

wandb: Adding directory to artifact (./data/simple_wiki_split_long)... Done. 0.9s


# Make dataset

In [8]:
simple_wiki_split_long = load_from_disk('./data/simple_wiki_split_long')

In [6]:
openai.api_key = getpass('Enter your OpenAI key: ')

Enter your OpenAI key: ··········


In [ ]:
model_engine = 'gpt-3.5-turbo'

message = 'You are a helpful assistant that generates questions from text.'
prompt_template = ("Question: X\nAnswer:", "\nWhat kind of question, X, could this be an answer to?\nX:")

make_instruct_dataset(simple_wiki_split_long,
                          message=message,
                          prompt_template=prompt_template,
                          split='validation',
                          text_key='trunc_text',
                          length=10**3,
                          sleep_timer=1,
                          file_name='df_simple_wiki_long_qa')

In [ ]:
model_engine = 'gpt-3.5-turbo'

message = 'You are a helpful assistant that generates questions from text.'
prompt_template = ("Question: X\nAnswer:", "\nWhat kind of question, X, could this be an answer to?\nX:")

make_instruct_dataset(simple_wiki_split_long,
                          message=message,
                          prompt_template=prompt_template,
                          split='test',
                          text_key='trunc_text',
                          length=10**3,
                          sleep_timer=2,
                          file_name='df_simple_wiki_long_qa')

In [ ]:
model_engine = 'gpt-3.5-turbo'

message = 'You are a helpful assistant that generates questions from text.'
prompt_template = ("Question: X\nAnswer:", "\nWhat kind of question, X, could this be an answer to?\nX:")

make_instruct_dataset(simple_wiki_split_long,
                          message=message,
                          prompt_template=prompt_template,
                          split='train',
                          text_key='trunc_text',
                          length=10**4,
                          sleep_timer=0,
                          file_name='df_simple_wiki_long_qa')

In [52]:
df_answers = {}
df_questions ={}
df_qa = {}

for split in ['train','validation','test']:

    df_answers[split] = pd.DataFrame(simple_wiki_split_long[split])
    df_answers[split]['id']=df_answers[split]['id'].astype(int)
    df_answers[split].set_index('id',inplace=True)

    df_questions[split] = pd.read_csv(f'./data/df_simple_wiki_long_qa_{split}.csv',
                                        index_col='Unnamed: 0')

    df_questions[split]['id']=df_questions[split]['id'].astype(int)
    df_questions[split].set_index('id',inplace=True)

    df_qa[split] = df_questions[split].join(df_answers[split][['trunc_text']],
                                                        how='left',on='id',
                                                        lsuffix='l',rsuffix='r')

    df_qa[split].to_csv(f'./data/simple_wiki_QA_combined_{split}.csv')

# Compare with English Wiki

In [ ]:
english_wiki = load_dataset("wikipedia", "20220301.en")

Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]